In [922]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy import interpolate
import h5py
import time

In [923]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [924]:
from bender_functions import Bender

In [925]:
bender = Bender()

# Set up movement and activation

## Details about the fish and prep

In [926]:
fishcode = "scup10"

Basic measurements for the fish

In [927]:
fishlen = 220       # mm
fishmass = 220      # g

Location of the bending point along the body. Distance from the head to posterior edge of the anterior clamp

In [928]:
dbend = 142          # mm

Distance between the two clamps:

In [929]:
dclamp = 19         # mm

Vertical and horizontal distance from the transducer to the center of pressure on the fish

In [930]:
dvert = 135          # mm
dhoriz = 0          # mm

Cross sectional size of the fish

In [931]:
xsec_width = 20          # mm
xsec_height = 60         # mm

## Output file

Remember to use double backslashes in the file name

In [932]:
outputfile = 'C:\\Data\\scup\\2021-06-18\\trial_001.h5'
outputfile = bender.increment_file_name(outputfile)

print('Actual output file: {}'.format(outputfile))

Actual output file: C:\Data\scup\2021-06-18\trial_024.h5


## Movement parameters

In [933]:
freq = 5     # Hz
curve = 5.0       # 1/m
amp = np.rad2deg(curve * (dclamp/1000))

print("Amplitude for {} 1/m curvature is {} deg".format(curve, amp))

ncycles = 10

scale = 6       # output teeth divided by input teeth

waitbefore = 3.0
waitafter = 2.0

Amplitude for 5.0 1/m curvature is 5.443099053742821 deg


## Activation parameters

Remember to set the activation pulse rate on the S88 S1 rate dial

In [934]:
is_activation = True        # set to False for passive tests

activation_duty = 0.25         # fractions of a cycle
activation_phase = -0.13      # fractions of a cycle

activation_pulse_rate = 75  # Hz
start_cycle = 3

In [935]:
actburstdur = activation_duty / freq

# make sure the activation is an even number of pulses
actburstdur = np.floor(actburstdur * activation_pulse_rate * 2) / (
        activation_pulse_rate * 2)

actburstduty = actburstdur * freq

print("Activation burst duration: {:.3} msec".format(actburstdur*1000))
print("True activation duty: {:.3}".format(actburstduty))

Activation burst duration: 86.7 msec
True activation duty: 0.433


Remember to set the S88 train duration dial to the activation burst duration above!

The parameters below are set on the S88 front panel. Make sure you record the correct values!

In [936]:
S1volts = 11
S2volts = 1.5
S1pulsedur = 2          # ms
S2pulsedur = 2          # ms
S1side = 'left'
S2side = 'right'

# Sampling parameters and channels

Sampling parameters

In [937]:
samplefreq = 1000.0
outputfreq = 100000.0

In [938]:
device_name = '/Dev1'

## Analog output channel

Sends the pulses to the S88 for muscle activation

In [939]:
bender.set_activation_channels('ao0', 'ao1')

## Digital output channel

Controls the motor

In [940]:
bender.set_motor_channel('port0')

## Analog input channels

Six channels from the force transducer, plus the monitor channel from the S88 stimulator.

In [941]:
SG0_chan = 'ai0'
SG1_chan = 'ai1'
SG2_chan = 'ai2'
SG3_chan = 'ai3'
SG4_chan = 'ai4'
SG5_chan = 'ai5'

In [942]:
activation_monitor_chan = 'ai6'

In [943]:
inchannels = [SG0_chan, SG1_chan, SG2_chan, SG3_chan, SG4_chan, SG5_chan,
                activation_monitor_chan]
inchannel_names = ['SG0', 'SG1', 'SG2', 'SG3', 'SG4', 'SG5',
                    'activation_monitor']

bender.set_input_channels(inchannels, inchannel_names)

Force transducer calibration file

In [944]:
bender.loadCalibration('FT17161.cal')
bender.calibration

array([[ 9.609000e-02, -2.877000e-01,  1.039313e+01, -2.850000e-03,
        -1.706000e-01, -2.490000e-03],
       [ 5.423000e-02, -7.349830e+00,  2.909400e-01, -4.012000e-02,
        -4.740000e-03, -8.726000e-02],
       [-7.387000e-02,  8.691000e-02,  1.065666e+01,  1.484400e-01,
         8.860000e-02,  0.000000e+00],
       [ 6.261130e+00,  3.686710e+00, -4.922800e-01,  1.253000e-02,
        -3.947000e-02, -8.481000e-02],
       [-7.690000e-03, -6.688000e-02,  1.045477e+01, -1.517300e-01,
         8.331000e-02,  1.500000e-04],
       [-6.235270e+00,  3.543590e+00, -3.817600e-01,  2.526000e-02,
         3.202000e-02, -8.559000e-02]])

## Encoder angle input

In [945]:
encoder_counts_per_rev = 10000
bender.set_encoder_channel('ctr0', counts_per_rev=encoder_counts_per_rev)

Start setting up the output

In [946]:
movedur = ncycles / freq
totaldur = waitbefore + movedur + waitafter

t = np.arange(0, totaldur, 1.0/samplefreq)
t -= waitbefore

tnorm = t * freq

Generate the angle and angular velocity signals

In [947]:
rampdur = 0.25

In [948]:
angle = amp * np.sin(2*np.pi * freq * t)
anglevel = 2*np.pi * amp * freq * np.cos(2*np.pi * freq * t)

angle[t < 0] = 0
angle[t > movedur] = 0

anglevel[t < 0] = 0
anglevel[t > movedur] = 0

Ramp to the amplitude

In [949]:
rampvel = amp / rampdur
tendramp1 = 0.25 / freq
tstartramp1 = tendramp1 - rampdur

tstartramp2 = (ncycles - 0.25) / freq
tendramp2 = tstartramp2 + rampdur

rampangle1 = (t[(t >= tstartramp1) & (t < tendramp1)] - tstartramp1) * rampvel
rampangle2 = (t[(t >= tstartramp2) & (t < tendramp2)] - tendramp2) * rampvel

np.place(angle, (t >= tstartramp1) & (t < tendramp1), rampangle1)
np.place(anglevel, (t >= tstartramp1) & (t < tendramp1), rampvel)

np.place(angle, (t >= tstartramp2) & (t < tendramp2), rampangle2)
np.place(anglevel, (t >= tstartramp2) & (t < tendramp2), rampvel)

In [950]:
bender.set_bending_signal(t, angle, anglevel)

In [951]:
S1actcmd = np.zeros_like(t)
S2actcmd = np.zeros_like(t)
Lonoff = []
Ronoff = []

if is_activation:
    pulsedur = 0.01         # 10ms long pulse to start the S88

    startstim = np.zeros((int(actburstdur * samplefreq + 1),))
    startstim[:int(pulsedur * samplefreq)] = 5

    actpulsephase = t[(t >= 0) & (t < actburstdur)] * activation_pulse_rate
    S2burst = (np.mod(actpulsephase, 1) <= 0.5).astype(float)
    S2burst *= 5.0

    bendphase = tnorm - 0.25

    # list of cycles when we'll have activation
    actcycles = list(range(start_cycle, ncycles))

    # also do one cycle before the bending starts
    actcycles.insert(0, -3)
    # and one after
    actcycles.append(ncycles+1)

    for c in actcycles:
        k = np.argmax(bendphase >= c + activation_phase)
        tstart = t[k]
        tend = tstart + actburstdur

        if any(bendphase >= c + activation_phase):
            Lonoff.append([tstart, tend])
        if any(bendphase >= c + activation_phase + 0.5):
            Ronoff.append(np.array([tstart, tend]) + 0.5 / freq)

        np.place(S1actcmd, (bendphase >= c + activation_phase) &
                          (bendphase < c + activation_phase + actburstduty),
                            startstim)
        np.place(S2actcmd, (bendphase >= c + 0.5 + activation_phase) &
                            (bendphase < c + 0.5 + activation_phase + actburstduty),
                            S2burst)

Lonoff = np.array(Lonoff)
Ronoff = np.array(Ronoff)

In [952]:
bender.set_activation(S1actcmd, S2actcmd)

and plot them:

In [953]:
fig = make_subplots(rows = 2, cols = 1,
                   shared_xaxes=True)
fig.add_trace(
    go.Scatter(x = t, y = angle, mode="lines", name="angle"),
    row=1, col=1)
fig.add_trace(
    go.Scatter(x = t, y = S1actcmd, mode="lines", name="S1"),
    row=1, col=1)
fig.add_trace(
    go.Scatter(x = t, y = S2actcmd, mode="lines", name="S2"),
    row=1, col=1)

for onoff in Lonoff:
    fig.add_vrect(x0 = onoff[0], x1=onoff[1], fillcolor="black", opacity=0.25, line_width=0,
                      row=1, col=1)

for onoff in Ronoff:
    fig.add_vrect(x0 = onoff[0], x1=onoff[1], opacity=0.7, line_width=1,
                      row=1, col=1)

fig.update_yaxes(title_text = "angle (deg)", row=1)
fig.add_trace(
    go.Scatter(x = t, y = anglevel, mode="lines", name="anglevel"),
    row=2, col=1)

fig.update_yaxes(title_text = "angular velocity (deg/s)", row=2)
fig.update_xaxes(title_text = "time (s)", row=2)

Generate the motor step and direction pulses. 

In [954]:
tout, dig, step, direction = bender.make_motor_stepper_pulses(outputfreq,
                        scale=scale,
                        stepsperrev=1600)

Use the cell below to debug the step and direction pulses, but don't render it every time. It takes a long time to plot the traces, because the output sampling rate is high.

In [955]:
# fig = make_subplots()
# fig.add_trace(go.Scatter(x=tout, y=step, mode="lines", name="step"))
# fig.add_trace(go.Scatter(x=tout, y=direction, mode="lines", name="dir"))
# fig.add_trace(go.Scatter(x=tout, y=dig, mode="lines", name="port"))

# Do data acquisition

## Main code block

Sets up the DAQ, sends the output, records the input, and writes it to the file.

In [956]:
aidata = bender.run(device_name)


In [957]:
forcetorque = bender.applyCalibration(aidata)
forcetorque_names = ['xForce', 'yForce', 'zForce', 'xTorque', 'yTorque', 'zTorque']

In [958]:
angle_measured = bender.angle

In [959]:
with h5py.File(outputfile, 'w') as f:
    f.attrs['EndTime'] = bender.endTime.strftime('%Y-%m-%d %H:%M:%S %Z')
    f.attrs['FishCode'] = fishcode
    f.attrs['FishLength_mm'] = fishlen
    f.attrs['FishMass_g'] = fishmass
    f.attrs['FishCrossSectionWidth_mm'] = xsec_width
    f.attrs['FishCrossSectionHeight_mm'] = xsec_height

    f.attrs['BendLocation_mm'] = dbend
    f.attrs['ClampDistance_mm'] = dclamp
    f.attrs['DistanceFromTransducerVert_mm'] = dvert
    f.attrs['DistanceFromTransducerHoriz_mm'] = dhoriz
    
    gin = f.create_group('RawInput')
    gin.attrs['SampleFrequency'] = samplefreq
    gin.create_dataset('forcetransducer', data=aidata[:6,:])
    gin.create_dataset('activation_monitor', data=aidata[6,:])

    gcal = f.create_group('Calibrated')
    for ft1, name1 in zip(forcetorque, forcetorque_names):
        gcal.create_dataset(name1, data=ft1)
    gcal.create_dataset('CalibrationMatrix', data=bender.calibration)

    ds = gcal.create_dataset('Encoder', data=bender.angledata)
    ds.attrs['CountsPerRev'] = encoder_counts_per_rev

    # save the output data
    gout = f.create_group('Output')
    gout.attrs['SampleFrequency'] = outputfreq
    gout.create_dataset('DigitalOut', data=dig)
    gout.create_dataset('SyncInTrainDur', data=S1actcmd)
    gout.create_dataset('SyncInS2Del', data=S2actcmd)
    gout.attrs['S1side'] = S1side
    gout.attrs['S2side'] = S2side
    gout.attrs['S1volts'] = S1volts
    gout.attrs['S2volts'] = S2volts
    gout.attrs['S1pulsedur_ms'] = S1pulsedur    
    gout.attrs['S2pulsedur_ms'] = S2pulsedur    
    
    # save the parameters for generating the stimulus
    gout = f.create_group('NominalStimulus')
    gout.attrs['Type'] = 'Constant Frequency'

    gout.create_dataset('t', data=t)
    ds = gout.create_dataset('Position', data=angle)
    ds.attrs['Units'] = 'deg'
    ds = gout.create_dataset('Velocity', data=anglevel)
    ds.attrs['Units'] = 'deg/sec'
    gout.create_dataset('tnorm', data=tnorm)
    gout.create_dataset('Lonoff', data=Lonoff)
    gout.create_dataset('Ronoff', data=Ronoff)

    gout.attrs['Amplitude'] = amp
    gout.attrs['Curvature'] = curve
    gout.attrs['Frequency'] = freq
    gout.attrs['Cycles'] = ncycles
    gout.attrs['WaitPre'] = waitbefore
    gout.attrs['WaitPost'] = waitafter
    gout.attrs['ScaleFactor'] = scale

    gout.attrs['ActivationOn'] = is_activation
    gout.attrs['ActivationDuty'] = activation_duty
    gout.attrs['ActivationStartPhase'] = activation_phase
    gout.attrs['ActivationStartCycle'] = start_cycle
    


In [960]:
fig = make_subplots(rows = 2, cols = 1,
                   shared_xaxes=True)
fig.add_trace(
    go.Scatter(x = t, y = angle_measured, mode="lines", name="angle"),
    row=1, col=1)
fig.add_trace(
    go.Scatter(x = t, y = aidata[6,:], mode="lines", name="stim"),
    row=1, col=1)
fig.add_trace(
    go.Scatter(x = t, y = forcetorque[3,:], mode="lines", name="Tx"),
    row=2, col=1)

for onoff in Lonoff:
    fig.add_vrect(x0 = onoff[0], x1=onoff[1], fillcolor="black", opacity=0.25, line_width=0,
                        row="all", col="all")

for onoff in Ronoff:
    fig.add_vrect(x0 = onoff[0], x1=onoff[1], opacity=0.7, line_width=1,
                      row="all", col="all")

fig.update_yaxes(title_text = "angle (deg)", row=1)
fig.update_yaxes(title_text = "torque (Nm)", row=2)
fig.update_xaxes(title_text = "time (s)", row=2)


In [100]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=angle, y=forcetorque[3,:]))
fig.update_yaxes(title_text="torque (Nm)")
fig.update_xaxes(title_text="angle (deg)")